In [1]:
import os
os.environ["OMP_NUM_THREADS"]="4"
os.environ["MKL_NUM_THREADS"]="4"
os.environ["VECLIB_MAXIMUM_THREADS"]="4"
os.environ["NUMEXPR_NUM_THREADS"]="4"
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"

In [ ]:
import sys
import scanpy as sc

sys.path.append("../..")
from scripts.runner_models import cv_scvi_5fold



In [3]:
# Load data
IN_PATH_neu = "../data/raw_count/GSE169569_raw_counts.h5ad"
IN_PATH_cov = "../data/raw_count/GSE228841_raw_counts.h5ad"

adata_neu = sc.read_h5ad(IN_PATH_neu)
adata_cov = sc.read_h5ad(IN_PATH_cov)

In [4]:
import numpy as np

# Extract data matrices and metadata
X_counts = adata_neu.layers["counts"] # raw counts
gene_names = np.array(adata_neu.var_names, dtype=str)
batches = np.array(adata_neu.obs["BioProject"], dtype=str)
norm_layer = adata_neu.layers["log2_1p_CPM_original"]

In [9]:
# Optuna objective for scVI + helper to run study
import optuna
from functools import partial

def scvi_objective(
    trial,
    X_counts, gene_names, batches, norm_layer,
    k_search=3,
    seed=123,
    batch_key="BioProject",
    seurat_layer="log2_1p_CPM_original",
):
    """
    Objective:
      - Sample hyperparameters including n_hvg.
      - Run cv_scvi_5fold on a one-point grid.
      - Return NB_ll_thin mean as optimization target.
    """

    # Hyperparameter search space
    hp = dict(
        n_latent=trial.suggest_categorical("n_latent", [16, 32]),
        n_hidden=trial.suggest_categorical("n_hidden", [128, 256]),
        n_layers=trial.suggest_categorical("n_layers", [2]),
        dropout_rate=trial.suggest_categorical("dropout_rate", [0.0, 0.1]),
        max_epochs=trial.suggest_categorical("max_epochs", [150, 250]),
        lr=trial.suggest_categorical("lr", [1e-3, 5e-4]),
        weight_decay=trial.suggest_categorical("weight_decay", [1e-6, 1e-5]),
        batch_size=trial.suggest_categorical("batch_size", [256, 512]),
        n_hvg=trial.suggest_categorical("n_hvg", [800, 1000, 1500, 2000]),
        use_gpu=True,
    )

    # Convert to one-point grid
    grid = {k: [v] for k, v in hp.items() if k != "n_hvg"}
    print(grid)

    # Run CV (with HVG value from hp)
    summary, detailed_df = cv_scvi_5fold(
        X_counts,
        k=k_search,
        scvi_grid=grid,
        n_hvg=hp["n_hvg"],
        R=1,
        mask_frac=0.10,
        thinning_p=0.10,
        random_state=seed,
        hvg_mode="seurat_v3",
        gene_names=gene_names,
        batches=batches,
        norm_layer=norm_layer,
        batch_key=batch_key,
        seurat_layer_name=seurat_layer,
        labels=None,
        save_dir=None,
    )

    return summary, detailed_df, float(summary["NB_ll_thin"].mean())


In [10]:
# Run Optuna study
seed = 123
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=seed),
)

obj = partial(
    scvi_objective,
    X_counts=X_counts,
    gene_names=gene_names,
    batches=batches,
    norm_layer=norm_layer,
    k_search=3,          # small and quick
    seed=seed,
    batch_key="BioProject",
    seurat_layer="log2_1p_CPM_original",
)

study.optimize(obj, n_trials=20, gc_after_trial=True)

print("Best value:", study.best_value)
print("Best params:", study.best_params)
best = study.best_params

{'n_latent': [16], 'n_hidden': [256], 'n_layers': [2], 'dropout_rate': [0.0], 'max_epochs': [150], 'lr': [0.001], 'weight_decay': [1e-05], 'batch_size': [256], 'use_gpu': [True]}
2025-09-15 09:31:24 | [cv_scvi_5fold] start k=3 n_hvg=1000 mode=seurat_v3
2025-09-15 09:31:24 | X_counts: dense  (447, 51777) dtype=int64
2025-09-15 09:31:24 | X_counts: zeros=66.23%, approx_int=True
2025-09-15 09:31:24 | [time] [scVI] fold 1/3 start


/Users/mjlee/master_thesis/Thesis_project/thesispjt/thesis-impute/lib/python3.12/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  return fn(*args_all, **kw)


2025-09-15 09:31:25 | [scVI] fold 1 | HVG=1000
2025-09-15 09:31:25 | [scVI] fold 1 params={'batch_size': 256, 'dropout_rate': 0.0, 'lr': 0.001, 'max_epochs': 150, 'n_hidden': 256, 'n_latent': 16, 'n_layers': 2, 'use_gpu': True, 'weight_decay': 1e-05}
2025-09-15 09:31:25 | [scVI] fit hp={'n_latent': 16, 'n_hidden': 256, 'n_layers': 2, 'dropout_rate': 0.0, 'max_epochs': 150, 'lr': 0.001, 'weight_decay': 1e-05, 'batch_size': 256, 'batch_key': 'BioProject', 'use_gpu': True}
2025-09-15 09:31:25 | X_train: dense  (298, 1000) dtype=int64
2025-09-15 09:31:25 | [time] [scVI] train start


/Users/mjlee/master_thesis/Thesis_project/thesispjt/thesis-impute/lib/python3.12/site-packages/scvi/train/_trainrunner.py:82: UserWarning: `accelerator` has been set to `mps`. Please note that not all PyTorch/Jax operations are supported with this backend. as a result, some models might be slower and less accurate than usuall. Please verify your analysis!Refer to https://github.com/pytorch/pytorch/issues/77764 for more details.
  accelerator, lightning_devices, device = parse_device_args(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/mjlee/master_thesis/Thesis_project/thesispjt/thesis-impute/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/mjlee/master_thesis/Thesis_p

Training:   0%|          | 0/150 [00:00<?, ?it/s]

...

/Users/mjlee/master_thesis/Thesis_project/thesispjt/thesis-impute/lib/python3.12/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  return fn(*args_all, **kw)
/Users/mjlee/master_thesis/Thesis_project/thesispjt/thesis-impute/lib/python3.12/site-packages/scvi/train/_trainrunner.py:82: UserWarning: `accelerator` has been set to `mps`. Please note that not all PyTorch/Jax operations are supported with this backend. as a result, some models might be slower and less accurate than usuall. Please verify your analysis!Refer to https://github.com/pytorch/pytorch/issues/77764 for more details.
  accelerator, lightning_devices, device = parse_device_args(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/mjlee/master_thesis/Thesis_project/thesispjt/thesis-impute/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.p

Training:   0%|          | 0/150 [00:00<?, ?it/s]

2025-09-15 09:32:10 | [time] [scVI] train end in 0.19s
2025-09-15 09:32:10 | [scVI] predict_mean
2025-09-15 09:32:10 | [time] [scVI] get_likelihood_parameters start
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2025-09-15 09:32:10 | [time] [scVI] get_likelihood_parameters end in 0.01s
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2025-09-15 09:32:11 | [scVI] predict_mean
2025-09-15 09:32:11 | [time] [scVI] get_likelihood_parameters start
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2025-09-15 09:32:11 | [time] [scVI] get_likelihood_parameters end in 0.01s
2025-09-15 09:32:11 | [scVI] predict_mean
2025-09-15 09:32:11 | [time] [scVI] get_likelihood_parameters start
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2

[W 2025-09-15 09:32:11,074] Trial 19 failed with parameters: {'n_latent': 32, 'n_hidden': 128, 'n_layers': 2, 'dropout_rate': 0.0, 'max_epochs': 150, 'lr': 0.0005, 'weight_decay': 1e-06, 'batch_size': 512, 'n_hvg': 1500} because of the following error: The value   model  n_hvg     MAE_zero      MSE_zero  MedianL1_zero  NB_ll_zero  \
0  scVI   1500  1076.401045  1.616236e+08     308.089732   -7.732224   

   NB_dev_zero   MAE_thin       MSE_thin  MedianL1_thin  NB_ll_thin  \
0     6.597972  67.444219  773826.232447      29.615018   -2.882951   

   NB_dev_thin Silhouette                                             params  
0     4.003254        NaN  {'batch_size': 512, 'dropout_rate': 0.0, 'lr':...   could not be cast to float.
[W 2025-09-15 09:32:11,077] Trial 19 failed with value (  model  n_hvg     MAE_zero      MSE_zero  MedianL1_zero  NB_ll_zero  \
0  scVI   1500  1076.401045  1.616236e+08     308.089732   -7.732224   

   NB_dev_zero   MAE_thin       MSE_thin  MedianL1_thin  NB_ll

ValueError: No trials are completed yet.

In [ ]:
# Final retrain/eval using the best HPs from the study
# Uses a larger k, more epochs/R, and writes CSVs.

# Build one-point grid from best params
best_grid = {k: [best[k]] for k in best}

final_summary, final_details = cv_scvi_5fold(
    X_counts,
    k=5,                     # fuller CV
    scvi_grid=best_grid,
    n_hvg=int(1000),         # or tune via Optuna if you prefer
    R=3,                     # more repeats for stability
    mask_frac=0.10,
    thinning_p=0.10,
    random_state=seed,
    hvg_mode="seurat_v3",
    gene_names=gene_names,
    batches=batches,
    norm_layer=norm_layer,
    batch_key="BioProject",
    seurat_layer_name="log2_1p_CPM_original",
    labels=None,
    save_dir="results/scvi_full",   # CSVs will be written here
    heartbeat_path=None,
)

display(final_summary.head())
display(final_details.head())
print("Saved to: results/scvi_full")


In [8]:
final_summary

NameError: name 'final_summary' is not defined